In [1]:
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer

from sklearn.preprocessing import OneHotEncoder

In [2]:
#utilizo la funcion de fernando pero sin encodiar

def limpiar_DF(df):
    # inicialmente voy con el tratamiento mas simple sobre los datos faltantes
    df = df.fillna(0)
    df = df.dropna()
    '''
      removemos la columna de la cual el modelo podria aprender que su existencia
      implica el Closed_won (ya que es un valor que se obtiene a posteriori de haber
      ganado y estaria mal usarlo ya que los modelos entrenadas con ella serian 
      incapaces de poder predecir correctamente)
    '''
    df = df.drop(['Sales_Contract_No'], axis=1)
    
    # irrelevantes
    df = df.drop(['Opportunity_Name'], axis=1)

    # dato cte = NaT
    df = df.drop(['Last_Activity'], axis=1)

    # basado en lo charlado con el grupo, ahora subdividimos el df de forma de tener
    # en cuenta los casos cerrados y cobertimos las varaibles categorias en su
    # representacion mas simple (siendo dos casos en Cloased_Won = 1 y Closed:_lost = 0)
    if 'Stage' in df.columns:
        df = df[((df['Stage'] == 'Closed Won') | (df['Stage'] == 'Closed Lost'))]
        df['Stage'] = df['Stage'].apply(lambda x: 1 if x == 'Closed Won' else 0)
        
    return df

In [3]:



def normaliza_df(df, ones_hots):
    # inicialmente voy con el tratamiento mas simple sobre los datos faltantes
    df = df.fillna(0)
    df = df.dropna()
    '''
      removemos la columna de la cual el modelo podria aprender que su existencia
      implica el Closed_won (ya que es un valor que se obtiene a posteriori de haber
      ganado y estaria mal usarlo ya que los modelos entrenadas con ella serian 
      incapaces de poder predecir correctamente)
    '''
    df = df.drop(['Sales_Contract_No'], axis=1)
    
    # irrelevantes
    df = df.drop(['Opportunity_Name'], axis=1)

    # dato cte = NaT
    df = df.drop(['Last_Activity'], axis=1)

    # basado en lo charlado con el grupo, ahora subdividimos el df de forma de tener
    # en cuenta los casos cerrados y cobertimos las varaibles categorias en su
    # representacion mas simple (siendo dos casos en Cloased_Won = 1 y Closed:_lost = 0)
    if 'Stage' in df.columns:
        df = df[((df['Stage'] == 'Closed Won') | (df['Stage'] == 'Closed Lost'))]
        df['Stage'] = df['Stage'].apply(lambda x: 1 if x == 'Closed Won' else 0)
    
    # hacemos que las variables temporales en las que nos vanos a enfocar sean del tipo correcto
    df['Account_Created_Date'] = pd.to_datetime(df['Account_Created_Date'], format="%m/%d/%Y")
    df['Opportunity_Created_Date'] = pd.to_datetime(df['Opportunity_Created_Date'], format="%m/%d/%Y")

    df['Quote_Type'] = df['Quote_Type'].apply(lambda x: 1 if x == 'Binding' else 0)
    
    # todo: convertir estas en una ventana de tiempo
    df = df.drop(['Planned_Delivery_Start_Date'], axis=1)
    df = df.drop(['Planned_Delivery_End_Date'], axis=1)

    categoric_cols = df.columns[df.dtypes==object].tolist() 
    numeric_cols = df.columns[df.dtypes=='float64'].tolist() 
    numeric_cols_2 = df.columns[df.dtypes=='int64'].tolist() 
    date_cols = df.columns[df.dtypes=='datetime64[ns]'].tolist() 
    
    print(categoric_cols)
    print('--------------')
    
    
    def crearOneHotDF(c1):
        if 'Stage' in df.columns:
            onehotencoder = OneHotEncoder(handle_unknown = 'ignore')
            onehotencoder.fit(df[[c1]])
            ones_hots[c1] = onehotencoder
        else:
            onehotencoder = ones_hots[c1]
        
        M = pd.DataFrame(onehotencoder.transform(df[[c1]]).toarray())
            
        num_cols = len(list(M))
        rng = range(0, num_cols)
        new_cols = [c1 + str(i) for i in rng] 
        M.columns = new_cols[:num_cols]
        return M

    categorical = pd.concat([crearOneHotDF(i) for i in categoric_cols], axis=1)

    df = pd.concat([categorical, df[numeric_cols], df[numeric_cols_2], df[date_cols]], axis=1)
    
    df = df.sort_values(by="Opportunity_Created_Date")
    
    df = df.drop(columns = 'Opportunity_Created_Date')
    df = df.drop(columns = 'Account_Created_Date')

    df = df.fillna(0)
    df = df.dropna()
    
    return df

In [4]:
train = pd.read_csv('/home/leandro/Documentos/Organizacion de datos/Fiuba-Big-Data-Analytics-TPs/Tp 2/Datos/Train_TP2_Datos_2020-2C.csv')

In [5]:
oneHot = {}
train_encodiado = normaliza_df(train,oneHot)
train_encodiado

['Region', 'Territory', 'Bureaucratic_Code', 'Source ', 'Billing_Country', 'Account_Name', 'Account_Owner', 'Opportunity_Owner', 'Account_Type', 'Opportunity_Type', 'Delivery_Terms', 'Brand', 'Product_Type', 'Size', 'Product_Category_B', 'Price', 'Currency', 'Quote_Expiry_Date', 'Last_Modified_Date', 'Last_Modified_By', 'Product_Family', 'Product_Name', 'ASP_Currency', 'ASP_(converted)_Currency', 'Month', 'Delivery_Quarter', 'Actual_Delivery_Date', 'Total_Amount_Currency', 'Total_Taxable_Amount_Currency', 'Prod_Category_A']
--------------


,Region0,Region1,Region2,Region3,Region4,Territory0,Territory1,Territory2,Territory3,Territory4,...,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,Submitted_for_Approval,Opportunity_ID,Quote_Type,Delivery_Year,TRF,Stage
16104,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,12369.0,0.0,2018.0,2.0,0.0
16105,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,0.0,12370.0,0.0,2016.0,20.0,0.0
16106,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,12371.0,0.0,2016.0,0.0,0.0
16107,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,12371.0,0.0,2016.0,0.0,0.0
16109,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,12372.0,0.0,2016.0,3.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16088,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16089,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16093,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16094,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Aplico lightgbm

# # Primero solo uso los valores numericos

In [6]:
# Separo en train y test el df
train_limpio= limpiar_DF(train)
train_limpio = train_limpio.select_dtypes(exclude=['object'])

import re
train_limpio = train_limpio.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))


x_train, x_test, y_train, y_test = train_test_split(train_limpio.drop(columns = 'Stage'), train_limpio['Stage'], test_size = 0.25, random_state = 0)


In [7]:
d_train = lgb.Dataset(x_train, label=y_train)
params = {}
params['learning_rate'] = 0.003
params['boosting_type'] = 'gbdt'
params['objective'] = 'binary'
params['metric'] = 'binary_logloss'
params['sub_feature'] = 0.5
params['num_leaves'] = 10
params['min_data'] = 50
params['max_depth'] = 10
clf = lgb.train(params, d_train, 100)

[LightGBM] [Info] Number of positive: 7206, number of negative: 5456
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000483 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1496
[LightGBM] [Info] Number of data points in the train set: 12662, number of used features: 12
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.569104 -> initscore=0.278198
[LightGBM] [Info] Start training from score 0.278198


In [8]:
#Prediction
y_pred=clf.predict(x_test)


In [9]:
test = pd.read_csv('/home/leandro/Documentos/Organizacion de datos/Fiuba-Big-Data-Analytics-TPs/Tp 2/Datos/Test_TP2_Datos_2020-2C.csv')
test_limpio = limpiar_DF(test)
val_X = test_limpio.select_dtypes(exclude=['object'])

In [10]:
predicts = clf.predict(val_X)
predicts

array([0.55856323, 0.58102699, 0.56376106, ..., 0.58922378, 0.5790067 ,
       0.50431765])

In [11]:
output = pd.DataFrame({'Opportunity_ID':val_X.Opportunity_ID, 'Target': predicts})
output

,Opportunity_ID,Target
0,10689,0.558563
1,10689,0.581027
2,10689,0.563761
3,10690,0.525597
4,10690,0.557976
...,...,...
2546,12364,0.587247
2547,12365,0.600166
2548,12366,0.589224
2549,12367,0.579007


In [12]:
output = output.groupby('Opportunity_ID').mean()
output

,Target
Opportunity_ID,
10689,0.567784
10690,0.551704
10691,0.616723
10692,0.530750
10693,0.610176
...,...
12364,0.597310
12365,0.600166
12366,0.589224


In [13]:
output.to_csv('/home/leandro/Documentos/Organizacion de datos/Fiuba-Big-Data-Analytics-TPs/Tp 2/Resultados/LightGBMScore1.csv')

# # Ahora pruebo con el one hot encoding realizado por fernando

In [14]:
# Separo en train y test el df

# Me saltaba error y buscando encontre que la siguiente linea solucionaba el error
import re
train_encodiado = train_encodiado.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))


x_train, x_test, y_train, y_test = train_test_split(train_encodiado.drop(columns = 'Stage'), train_encodiado['Stage'], test_size = 0.25, random_state = 0)


In [15]:
d_train = lgb.Dataset(x_train, label=y_train)
params = {}
params['learning_rate'] = 0.003
params['boosting_type'] = 'gbdt'
params['objective'] = 'binary'
params['metric'] = 'binary_logloss'
params['sub_feature'] = 0.5
params['num_leaves'] = 10
params['min_data'] = 50
params['max_depth'] = 10
clf = lgb.train(params, d_train, 100)

[LightGBM] [Info] Number of positive: 7184, number of negative: 5526
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.006617 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2389
[LightGBM] [Info] Number of data points in the train set: 12710, number of used features: 452
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.565224 -> initscore=0.262392
[LightGBM] [Info] Start training from score 0.262392


In [16]:
#Prediction
y_pred=clf.predict(x_test)

In [17]:
test_encodiado = normaliza_df(test,oneHot)
val_X = test_encodiado.copy()

['Region', 'Territory', 'Bureaucratic_Code', 'Source ', 'Billing_Country', 'Account_Name', 'Account_Owner', 'Opportunity_Owner', 'Account_Type', 'Opportunity_Type', 'Delivery_Terms', 'Brand', 'Product_Type', 'Size', 'Product_Category_B', 'Price', 'Currency', 'Quote_Expiry_Date', 'Last_Modified_Date', 'Last_Modified_By', 'Product_Family', 'Product_Name', 'ASP_Currency', 'ASP_(converted)_Currency', 'Month', 'Delivery_Quarter', 'Actual_Delivery_Date', 'Total_Amount_Currency', 'Total_Taxable_Amount_Currency', 'Prod_Category_A']
--------------


In [18]:
predicts = clf.predict(val_X)
predicts

array([0.6033993 , 0.53147998, 0.52574593, ..., 0.6033993 , 0.5883385 ,
       0.49452267])

In [19]:
output = pd.DataFrame({'Opportunity_ID':val_X.Opportunity_ID, 'Target': predicts})
output

,Opportunity_ID,Target
718,11222,0.603399
719,11223,0.531480
726,11224,0.525746
725,11224,0.525746
720,11223,0.531480
...,...,...
2545,12364,0.609522
2546,12364,0.594259
2547,12365,0.603399
2548,12366,0.588339


In [20]:
output = output.groupby('Opportunity_ID').mean()
output

,Target
Opportunity_ID,
10689,0.582387
10690,0.571499
10691,0.593675
10692,0.526537
10693,0.609687
...,...
12364,0.601890
12365,0.603399
12366,0.588339


In [21]:
output.to_csv('/home/leandro/Documentos/Organizacion de datos/Fiuba-Big-Data-Analytics-TPs/Tp 2/Resultados/LightGBMScore2.csv')

# Pruebo con Ordinal encoding

In [22]:
pip install ipynb

Note: you may need to restart the kernel to use updated packages.


In [23]:
from ipynb.fs.full.Funciones_utiles import ordinalEncoding

['Region', 'Territory', 'Bureaucratic_Code', 'Account_Created_Date', 'Source ', 'Billing_Country', 'Account_Name', 'Account_Owner', 'Opportunity_Owner', 'Account_Type', 'Opportunity_Type', 'Quote_Type', 'Delivery_Terms', 'Opportunity_Created_Date', 'Brand', 'Product_Type', 'Size', 'Product_Category_B', 'Price', 'Currency', 'Quote_Expiry_Date', 'Last_Modified_Date', 'Last_Modified_By', 'Product_Family', 'Product_Name', 'ASP_Currency', 'ASP_(converted)_Currency', 'Planned_Delivery_Start_Date', 'Planned_Delivery_End_Date', 'Month', 'Delivery_Quarter', 'Actual_Delivery_Date', 'Total_Amount_Currency', 'Total_Taxable_Amount_Currency', 'Prod_Category_A']
--------------


In [24]:
ordinal = {}
train_ordinal = ordinalEncoding(train, ordinal)


['Region', 'Territory', 'Bureaucratic_Code', 'Account_Created_Date', 'Source ', 'Billing_Country', 'Account_Name', 'Account_Owner', 'Opportunity_Owner', 'Account_Type', 'Opportunity_Type', 'Quote_Type', 'Delivery_Terms', 'Opportunity_Created_Date', 'Brand', 'Product_Type', 'Size', 'Product_Category_B', 'Price', 'Currency', 'Quote_Expiry_Date', 'Last_Modified_Date', 'Last_Modified_By', 'Product_Family', 'Product_Name', 'ASP_Currency', 'ASP_(converted)_Currency', 'Planned_Delivery_Start_Date', 'Planned_Delivery_End_Date', 'Month', 'Delivery_Quarter', 'Actual_Delivery_Date', 'Total_Amount_Currency', 'Total_Taxable_Amount_Currency', 'Prod_Category_A']
--------------


In [25]:
x_train, x_test, y_train, y_test = train_test_split(train_ordinal.drop(columns = 'Stage'), train_ordinal['Stage'], test_size = 0.25, random_state = 0)


In [26]:
d_train = lgb.Dataset(x_train, label=y_train)
params = {}
params['learning_rate'] = 0.003
params['boosting_type'] = 'gbdt'
params['objective'] = 'binary'
params['metric'] = 'binary_logloss'
params['sub_feature'] = 0.5
params['num_leaves'] = 10
params['min_data'] = 50
params['max_depth'] = 10
clf = lgb.train(params, d_train, 100)

LightGBMError: Do not support special JSON characters in feature name.